<a href="https://colab.research.google.com/github/wylhtydtm/Nematode-project/blob/master/McFly_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import h5py
import pandas as pd
import tensorflow as tf
import tables
from sklearn import preprocessing
from numpy import dstack
import tensorflow as tf

In [ ]:
!pip install tensorflow==2.0.0

In [ ]:
print(tf.__version__)

In [ ]:
!pip install mcfly
import mcfly

In [ ]:
hf = h5py.File('/content/drive/My Drive/McFly_test.hdf5', 'r')

In [ ]:
datasetnames= list(hf.keys())
print(datasetnames)


['MOA_0', 'MOA_1', 'MOA_10', 'MOA_11', 'MOA_12', 'MOA_13', 'MOA_17', 'MOA_2', 'MOA_23', 'MOA_3', 'MOA_4', 'MOA_6', 'MOA_7', 'MOA_9']


In [ ]:
data = []
with tables.File('/content/drive/My Drive/McFly_test.hdf5', 'r') as fid:
     for k in datasetnames:
      load_class = pd.DataFrame(fid.get_node('/'+k).read())
      data.append(load_class)
      
fid.close()

In [ ]:
data_total=pd.concat(data,axis=0, ignore_index=True)

In [ ]:
print(data_total)

In [ ]:
scaled_feat= data_total.copy()
feats=data_total[[1,2, 3,4, 5, 6, 7]]

In [ ]:
scaler = preprocessing.StandardScaler().fit(feats.values)
features = scaler.transform(feats.values)


In [ ]:
data_total.min()

In [ ]:
col_names=[1,2,3,4,5,6,7]
scaled_feat[col_names] = features
print(scaled_feat)

In [ ]:
def split_sequences( sequences, n_steps):
  X, y = list(), list()
  L= len(sequences)
  for i in range(0, L, n_steps):
    seq_x, seq_y = sequences.iloc[i:i+n_steps, 1:8], sequences.iloc[i,-1]
    X.append(seq_x)
    y.append(seq_y)
  return X, y
  

In [ ]:
data_input, data_output = split_sequences(data_total, 876)

In [ ]:
print(len(data_input))
print(len(data_output))

In [ ]:
data_input[0]

In [ ]:
from sklearn.utils import shuffle

In [ ]:
data_input, data_output=(shuffle(data_input, data_output))

In [ ]:
print(data_input[0])
data_input=dstack(data_input)

In [ ]:
data_input= data_input.transpose(2,0,1)

In [ ]:
data_input.shape
print(data_input[0:5].shape)

In [ ]:
labels_unique= list(set(data_output))
print(labels_unique)
print(len(data_output))

In [ ]:
train_size= 3280*0.75
val_size= 3280*0.85

X_train, X_val, X_test=data_input[0:2640],data_input[2640:2788], data_input[2788:3281]
y_train, y_val, y_test=data_output[0:2640],data_output[2640:2788], data_output[2788:3281]

In [ ]:
lb = preprocessing.LabelBinarizer()
lb.fit(labels_unique)
lb.classes_

In [ ]:
print(X_train.shape)
print(len(y_train))

(2640, 876, 7)
2640


In [ ]:
y_train_binary = lb.fit_transform(y_train)
y_val_binary = lb.fit_transform(y_val)
y_test_binary = lb.fit_transform(y_test)

num_classes = y_train_binary.shape[1]

In [ ]:
print(num_classes)
print(y_train_binary[0])

14
[0 0 0 1 0 0 0 0 0 0 0 0 0 0]


In [ ]:
#Mcfly create models by random research. We generate 8 models to keep the training short for the test
metric = 'accuracy'
models = mcfly.modelgen.generate_models(X_train.shape,
                                        number_of_classes=num_classes,
                                        number_of_models = 8,
                                        model_types=['CNN','DeepConvLSTM','InceptionTime'],
                                        metrics=[metric],
                                        deepconvlstm_max_conv_layers=2,
                                        deepconvlstm_min_lstm_dims=1,
                                        deepconvlstm_max_lstm_dims=5,
                                        IT_max_network_dept=4)


The value of model_types is set from ['CNN', 'DeepConvLSTM', 'ResNet', 'InceptionTime'] (default) to ['CNN', 'DeepConvLSTM', 'InceptionTime']
The value of metrics is set from ['accuracy'] (default) to ['accuracy']
The value of deepconvlstm_max_conv_layers is set from 10 (default) to 2
The value of deepconvlstm_min_lstm_dims is set from 10 (default) to 1
The value of deepconvlstm_max_lstm_dims is set from 100 (default) to 5


In [ ]:
# Inspect the model selected
models_to_print = range(len(models))
for i, item in enumerate(models):
    if i in models_to_print:
        model, params, model_types = item
        print("-------------------------------------------------------------------------------------------------------")
        print("Model " + str(i))
        print(" ")
        print("Hyperparameters:")
        print(params)
        print(" ")
        print("Model description:")
        model.summary()
        print(" ")
        print("Model type:")
        print(model_types)
        print(" ")

-------------------------------------------------------------------------------------------------------
Model 0
 
Hyperparameters:
{'learning_rate': 0.06387882555774206, 'regularization_rate': 0.0010564967583817973, 'filters': array([73, 96, 89, 35, 29, 81]), 'fc_hidden_nodes': 1741}
 
Model description:
Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_41 (Batc (None, 876, 7)            28        
_________________________________________________________________
conv1d_106 (Conv1D)          (None, 876, 73)           1606      
_________________________________________________________________
batch_normalization_42 (Batc (None, 876, 73)           292       
_________________________________________________________________
activation_37 (Activation)   (None, 876, 73)           0         
_________________________________________________________________
conv1d_107 (

In [ ]:
models

[(<tensorflow.python.keras.engine.sequential.Sequential at 0x7fe86b364390>,
  {'filters': [37],
   'learning_rate': 0.00017875020866105692,
   'lstm_dims': [1, 3],
   'regularization_rate': 0.001729798868006255},
  'DeepConvLSTM'),
 (<tensorflow.python.keras.engine.training.Model at 0x7fe8651a5b00>,
  {'filters_number': 66,
   'learning_rate': 0.0008364032548025711,
   'max_kernel_size': 48,
   'network_depth': 6,
   'regularization_rate': 0.0020284736624765487},
  'InceptionTime'),
 (<tensorflow.python.keras.engine.training.Model at 0x7fe864b1e898>,
  {'filters_number': 90,
   'learning_rate': 0.00048359183405530893,
   'max_kernel_size': 71,
   'network_depth': 5,
   'regularization_rate': 0.03086275195979217},
  'InceptionTime'),
 (<tensorflow.python.keras.engine.sequential.Sequential at 0x7fe864b1e668>,
  {'filters': [49, 28],
   'learning_rate': 0.0057722005670236215,
   'lstm_dims': [3, 5, 3, 4],
   'regularization_rate': 0.0002641829094025879},
  'DeepConvLSTM'),
 (<tensorflow.p

In [ ]:
from mcfly.find_architecture import train_models_on_samples
import os
outputfile = os.path.join('/content/drive/My Drive/modelcomparison.json')
histories, val_accuracies, val_losses = train_models_on_samples(X_train, y_train_binary,
                                                                X_val, y_val_binary,
                                                                models, nr_epochs=30,
                                                                subset_size=300,
                                                                early_stopping_patience = 5,
                                                                verbose=True,
                                                                outputfile=outputfile,
                                                                metric='accuracy')